In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import random
import tensorflow as tf
from tensorflow import keras

In [3]:
np.random.seed(42)


In [5]:
data=pd.read_csv('EURUSD_tick_OK-2.csv')

In [11]:
n_small=350000
data_a=np.array(data[:n_small])
N=50
L=n_small-N
data_b=np.zeros((L,N,5), dtype='float32')
Y=np.zeros((L,2),dtype='float32')

In [9]:
from tqdm import tqdm

In [13]:
for k in tqdm(range(n_small-N)):
    data_b[k,:,:]=data_a[k:k+N, :]
    Y[k,0]=data_a[k+N,3]
    Y[k,1]=data_a[k+N,1]

100%|██████████| 349950/349950 [00:01<00:00, 306146.11it/s]


In [15]:
train_start=0
train_end=int(np.floor(0.9*L))
train_end -=4955
test_start=train_end+1
test_end=L
X_train=np.zeros((train_end,N,5),dtype='float32')
X_test=np.zeros((test_end-train_end,N,5),dtype='float32')
y_train=Y[np.arange(train_start, train_end),:]
y_test=Y[np.arange(test_start,L),:]
data_add_row=np.array([y_test[L-test_start-1,0],y_test[L-test_start-1,1]])
y_test=np.vstack([y_test,data_add_row])

In [17]:
for z in tqdm(range(test_end)):
    if z<train_end:
        X_train[z,:,:]=data_b[z,:,:]
    else:
        X_test[z-train_end,:,:]=data_b[z,:,:]

100%|██████████| 349950/349950 [00:01<00:00, 346353.27it/s]


In [19]:
model=keras.Sequential([
    keras.layers.Conv1D(50, 5, padding='same',input_shape=(50, 5), activation=tf.nn.relu, kernel_initializer="normal"),
    keras.layers.MaxPooling1D(7),
    keras.layers.Conv1D(100, 5, padding='same', activation=tf.nn.relu, kernel_initializer="normal"),
    keras.layers.GlobalMaxPooling1D(),
    keras.layers.Dense(25,activation=tf.nn.relu,kernel_initializer="normal"),
    keras.layers.Dense(2)
])
print(model.summary())
    

C:\Users\SHIVASAI\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 50, 50)         │         1,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 50)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 7, 100)         │        25,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 100)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         2,525 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            52 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,977 (113.19 KB)

 Trainable params: 28,977 (113.19 KB)

 Non-trainable params: 0 (0.00 B)

None


In [21]:
model.compile(optimizer="adam",loss="mse",metrics=["mae"])

In [23]:
#student id  is 2338212
#so epochs is 1+2=
#batch_size=30
history = model.fit(X_train, y_train, batch_size=30, epochs=5, validation_split=0.2, verbose=1)

Epoch 1/5
8267/8267 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.0040 - mae: 0.0324 - val_loss: 8.9705e-04 - val_mae: 0.0206
Epoch 2/5
8267/8267 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 7.6626e-04 - mae: 0.0188 - val_loss: 8.9395e-04 - val_mae: 0.0208
Epoch 3/5
8267/8267 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 7.4131e-04 - mae: 0.0184 - val_loss: 8.5149e-04 - val_mae: 0.0195
Epoch 4/5
8267/8267 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 7.2959e-04 - mae: 0.0181 - val_loss: 8.2768e-04 - val_mae: 0.0191
Epoch 5/5
8267/8267 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 7.2171e-04 - mae: 0.0180 - val_loss: 8.1996e-04 - val_mae: 0.0191


In [25]:
mse, mae=model.evaluate(X_test,y_test, verbose=1)
print("Mean absolute error: %.5f" %mae)

1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0016 - mae: 0.0269
Mean absolute error: 0.02543


In [27]:
print("Mean absolute error in practical section : 0.02579")
print("Mean absolute error in logbook work      : %.5f" %mae)

Mean absolute error in practical section : 0.02579
Mean absolute error in logbook work      : 0.02543
